In [22]:
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import nn, optim

data_dir = "CUB_200_2011/CUB_200_2011/images"

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(128),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

subset_classes = ["001.Black_footed_Albatross", "002.Laysan_Albatross"]
datasets_dict = {
    x: datasets.ImageFolder(
        os.path.join(data_dir),
        transform=data_transforms[x]
    ) for x in ['train', 'val']
}

dataloaders = {
    x: DataLoader(datasets_dict[x], batch_size=64, shuffle=True, num_workers=2)
    for x in ['train', 'val']
}

dataset_sizes = {x: len(datasets_dict[x]) for x in ['train', 'val']}
class_names = datasets_dict['train'].classes

model = models.mobilenet_v2(pretrained=True)
num_ftrs = model.last_channel
model.classifier[1] = nn.Linear(num_ftrs, len(class_names))
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, criterion, optimizer, dataloaders, num_epochs=15):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

model = train_model(model, criterion, optimizer, dataloaders, num_epochs=5)

torch.save(model.state_dict(), 'bird_classifier_fast.pth')

print("Modèle entraîné et sauvegardé avec succès !")


Epoch 1/5
----------


KeyboardInterrupt: 

In [2]:
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import nn, optim

In [3]:
data_dir = "CUB_200_2011/CUB_200_2011/images"

In [15]:
# Ajustement du learning rate et simplification des augmentations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(128),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
subset_classes = ["001.Black_footed_Albatross", "002.Laysan_Albatross", "003.Sooty_Albatross", "004.Groove_billed_Ani"]

In [5]:
datasets_dict = {
    x: datasets.ImageFolder(
        os.path.join(data_dir),
        transform=data_transforms[x]
    ) for x in ['train', 'val']
}


In [18]:
# Création des DataLoaders
dataloaders = {
    x: DataLoader(datasets_dict[x], batch_size=64, shuffle=True, num_workers=2)
    for x in ['train', 'val']
}

# Taille des datasets
dataset_sizes = {x: len(datasets_dict[x]) for x in ['train', 'val']}
class_names = datasets_dict['train'].classes

# 3. Modèle MobileNetV2 avec fine-tuning
#model = models.mobilenet_v2(pretrained=True)
#num_ftrs = model.last_channel
#model.classifier[1] = nn.Linear(num_ftrs, len(class_names))
#
## Fine-tuning des couches convolutionnelles
#for param in model.features.parameters():
#    param.requires_grad = False  # Geler les poids des couches initiales
#
#model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Modèle (ResNet comme exemple)
from torchvision.models import resnet18
model = resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')


In [1]:
# 4. Définition de la fonction de perte et de l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)


NameError: name 'nn' is not defined

In [19]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [20]:
# 6. Fonction d'entraînement du modèle
def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=20):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Ajustement du learning rate
        scheduler.step()

    return model

In [21]:
# 7. Entraînement du modèle
model = train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=20)


Epoch 1/20
----------
train Loss: 5.5819 Acc: 0.0050
val Loss: 5.6783 Acc: 0.0050
Epoch 2/20
----------
train Loss: 5.5829 Acc: 0.0057
val Loss: 5.6728 Acc: 0.0050
Epoch 3/20
----------
train Loss: 5.5779 Acc: 0.0044
val Loss: 5.6713 Acc: 0.0048
Epoch 4/20
----------
train Loss: 5.5743 Acc: 0.0067
val Loss: 5.6792 Acc: 0.0048
Epoch 5/20
----------
train Loss: 5.5671 Acc: 0.0048
val Loss: 5.6713 Acc: 0.0045
Epoch 6/20
----------
train Loss: 5.5674 Acc: 0.0053
val Loss: 5.6761 Acc: 0.0048
Epoch 7/20
----------
train Loss: 5.5747 Acc: 0.0046
val Loss: 5.6717 Acc: 0.0040
Epoch 8/20
----------
train Loss: 5.5837 Acc: 0.0057
val Loss: 5.6765 Acc: 0.0045
Epoch 9/20
----------
train Loss: 5.5813 Acc: 0.0048
val Loss: 5.6856 Acc: 0.0047
Epoch 10/20
----------
train Loss: 5.5747 Acc: 0.0053
val Loss: 5.6689 Acc: 0.0046
Epoch 11/20
----------
train Loss: 5.5708 Acc: 0.0055
val Loss: 5.6704 Acc: 0.0049
Epoch 12/20
----------
train Loss: 5.5773 Acc: 0.0052
val Loss: 5.6688 Acc: 0.0052
Epoch 13/20
-

In [ ]:
# 8. Sauvegarde du modèle entraîné
torch.save(model.state_dict(), 'bird_classifier_finetuned.pth')

print("Modèle entraîné et sauvegardé avec succès !")

In [7]:
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import optim
from sklearn.model_selection import train_test_split

# Configuration
data_dir = "CUB_200_2011/CUB_200_2011/images"

# Transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(128),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Dataset
dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])
train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=0.3, random_state=42)
val_idx, test_idx = train_test_split(test_idx, test_size=0.33, random_state=42)

train_data = torch.utils.data.Subset(dataset, train_idx)
val_data = torch.utils.data.Subset(dataset, val_idx)
test_data = torch.utils.data.Subset(dataset, test_idx)

dataloaders = {
    'train': DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2),
    'test': DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)
}

# Modèle Personnalisé
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SimpleCNN(num_classes=len(dataset.classes))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# Optimisation
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Fonction d'entraînement
def train_model(model, criterion, optimizer, dataloaders, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss, running_corrects = 0.0, 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Entraîner et évaluer
model = train_model(model, criterion, optimizer, dataloaders, num_epochs=10)


Epoch 1/10
----------
train Loss: 5.5701 Acc: 0.0046
val Loss: 5.3103 Acc: 0.0030
Epoch 2/10
----------
train Loss: 5.3038 Acc: 0.0040
val Loss: 5.3156 Acc: 0.0030
Epoch 3/10
----------
train Loss: 5.3034 Acc: 0.0050
val Loss: 5.3182 Acc: 0.0017
Epoch 4/10
----------


KeyboardInterrupt: 

In [9]:
from collections import Counter
class_counts = Counter([dataset.targets[i] for i in range(len(dataset))])
print(f"Nombre d'images par classe : {class_counts}")


Nombre d'images par classe : Counter({0: 60, 1: 60, 3: 60, 9: 60, 10: 60, 12: 60, 13: 60, 20: 60, 24: 60, 25: 60, 26: 60, 28: 60, 29: 60, 30: 60, 34: 60, 35: 60, 37: 60, 39: 60, 40: 60, 41: 60, 43: 60, 44: 60, 45: 60, 46: 60, 47: 60, 48: 60, 49: 60, 50: 60, 51: 60, 52: 60, 53: 60, 54: 60, 55: 60, 56: 60, 58: 60, 60: 60, 61: 60, 62: 60, 63: 60, 65: 60, 66: 60, 67: 60, 68: 60, 69: 60, 70: 60, 71: 60, 72: 60, 73: 60, 75: 60, 76: 60, 78: 60, 79: 60, 80: 60, 81: 60, 82: 60, 84: 60, 85: 60, 86: 60, 87: 60, 88: 60, 89: 60, 90: 60, 91: 60, 92: 60, 93: 60, 94: 60, 95: 60, 97: 60, 98: 60, 99: 60, 101: 60, 102: 60, 103: 60, 105: 60, 107: 60, 108: 60, 109: 60, 110: 60, 111: 60, 113: 60, 115: 60, 117: 60, 119: 60, 120: 60, 121: 60, 122: 60, 126: 60, 127: 60, 128: 60, 129: 60, 130: 60, 131: 60, 132: 60, 133: 60, 135: 60, 136: 60, 137: 60, 138: 60, 139: 60, 141: 60, 142: 60, 143: 60, 144: 60, 145: 60, 146: 60, 147: 60, 149: 60, 151: 60, 153: 60, 154: 60, 155: 60, 157: 60, 158: 60, 160: 60, 161: 60, 1

In [18]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from collections import Counter

# Vérifiez si le GPU est disponible
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Chargement du dataset
data_dir = "CUB_200_2011/CUB_200_2011/images"
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Fixe une taille d'image
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Filtrer les 10 classes avec le plus d'images
class_counts = Counter(dataset.targets)
top_classes = [cls for cls, _ in class_counts.most_common(10)]
filtered_samples = [(path, label) for path, label in dataset.samples if label in top_classes]
class_mapping = {label: idx for idx, label in enumerate(top_classes)}
filtered_samples = [(path, class_mapping[label]) for path, label in filtered_samples]

class FilteredDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = datasets.folder.default_loader(path)
        if self.transform:
            image = self.transform(image)
        return image, label

filtered_dataset = FilteredDataset(filtered_samples, transform)

# Répartition Train/Validation/Test
train_size = int(0.7 * len(filtered_dataset))
val_size = int(0.15 * len(filtered_dataset))
test_size = len(filtered_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(filtered_dataset, [train_size, val_size, test_size])

# DataLoaders avec num_workers=0
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0),
    'test': DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)
}

# Définir un petit modèle CNN
class SmallCNN(nn.Module):
    def __init__(self, num_classes):
        super(SmallCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialisation du modèle
model = SmallCNN(num_classes=len(top_classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Fonction d'entraînement
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss, running_corrects = 0.0, 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
    return model

# Entraînement
if __name__ == "__main__":
    model = train_model(model, dataloaders, criterion, optimizer, num_epochs=10)


Epoch 1/10
----------
train Loss: 7.7692 Acc: 0.1095
val Loss: 2.3134 Acc: 0.1000
Epoch 2/10
----------
train Loss: 2.3046 Acc: 0.1095
val Loss: 2.3159 Acc: 0.0889
Epoch 3/10
----------
train Loss: 2.3049 Acc: 0.1071
val Loss: 2.3135 Acc: 0.1000
Epoch 4/10
----------
train Loss: 2.2177 Acc: 0.1690
val Loss: 2.3719 Acc: 0.1556
Epoch 5/10
----------
train Loss: 1.7066 Acc: 0.4310
val Loss: 2.4319 Acc: 0.1556
Epoch 6/10
----------
train Loss: 0.8143 Acc: 0.7690
val Loss: 3.8122 Acc: 0.1556
Epoch 7/10
----------
train Loss: 0.2993 Acc: 0.9095
val Loss: 6.8739 Acc: 0.1556
Epoch 8/10
----------
train Loss: 0.1218 Acc: 0.9786
val Loss: 7.0269 Acc: 0.1333
Epoch 9/10
----------
train Loss: 0.1389 Acc: 0.9690
val Loss: 8.8783 Acc: 0.1333
Epoch 10/10
----------
train Loss: 0.1983 Acc: 0.9762
val Loss: 7.2759 Acc: 0.1333


In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v2
from torch.utils.data import DataLoader, random_split
from collections import Counter

# Configuration du dispositif
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Préparer les données
data_dir = "CUB_200_2011/CUB_200_2011/images"

# Transformations avec augmentation des données
transform = transforms.Compose([
    transforms.RandomResizedCrop((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Charger le dataset initial
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Sélection des 10 classes avec le plus d'images
class_counts = Counter(dataset.targets)
top_classes = [cls for cls, _ in class_counts.most_common(10)]
filtered_samples = [(path, label) for path, label in dataset.samples if label in top_classes]
class_mapping = {label: idx for idx, label in enumerate(top_classes)}
filtered_samples = [(path, class_mapping[label]) for path, label in filtered_samples]

# Création d'un dataset filtré
class FilteredDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = datasets.folder.default_loader(path)
        if self.transform:
            image = self.transform(image)
        return image, label

filtered_dataset = FilteredDataset(filtered_samples, transform)

# Division des données en ensembles d'entraînement, validation et test
train_size = int(0.7 * len(filtered_dataset))
val_size = int(0.15 * len(filtered_dataset))
test_size = len(filtered_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(filtered_dataset, [train_size, val_size, test_size])

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0),
    'test': DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)
}

# Charger MobileNetV2 avec fine-tuning
model = mobilenet_v2(weights='IMAGENET1K_V1')
model.classifier[1] = nn.Linear(model.last_channel, len(top_classes))
model = model.to(device)

# Optimisation
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Fonction d'entraînement
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-' * 10)
        model.train()

        running_loss = 0.0
        running_corrects = 0

        # Phase d'entraînement
        for inputs, labels in dataloaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += (outputs.argmax(1) == labels).sum().item()

        epoch_loss = running_loss / len(dataloaders['train'].dataset)
        epoch_acc = running_corrects / len(dataloaders['train'].dataset)
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Phase de validation après chaque époque
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in dataloaders['val']:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += (outputs.argmax(1) == labels).sum().item()

        val_loss = val_loss / len(dataloaders['val'].dataset)
        val_acc = val_corrects / len(dataloaders['val'].dataset)
        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        scheduler.step()
    return model

# Fonction d'évaluation
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total

# Entraîner le modèle
model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=10)

# Évaluer sur le test
test_acc = evaluate_model(model, dataloaders['test'])
print(f"Test Accuracy: {test_acc:.4f}")


Epoch 1/10
----------
Train Loss: 1.7522 Acc: 0.3905
Val Loss: 1.2691 Acc: 0.5778
Epoch 2/10
----------
Train Loss: 1.1315 Acc: 0.5857
Val Loss: 1.1863 Acc: 0.5556
Epoch 3/10
----------
Train Loss: 1.0297 Acc: 0.6310
Val Loss: 0.9139 Acc: 0.6778
Epoch 4/10
----------
Train Loss: 0.8541 Acc: 0.6952
Val Loss: 1.0502 Acc: 0.5778
Epoch 5/10
----------
Train Loss: 0.9169 Acc: 0.6952
Val Loss: 1.0920 Acc: 0.6444
Epoch 6/10
----------
Train Loss: 0.7081 Acc: 0.7667
Val Loss: 0.9847 Acc: 0.6556
Epoch 7/10
----------
Train Loss: 0.8118 Acc: 0.7262
Val Loss: 0.9115 Acc: 0.7111
Epoch 8/10
----------
Train Loss: 0.6182 Acc: 0.7833
Val Loss: 0.9953 Acc: 0.6556
Epoch 9/10
----------
Train Loss: 0.5580 Acc: 0.8238
Val Loss: 1.0995 Acc: 0.6667
Epoch 10/10
----------
Train Loss: 0.5698 Acc: 0.8167
Val Loss: 0.7865 Acc: 0.7222
Test Accuracy: 0.7111


In [6]:
# Sauvegarder le modèle
torch.save(model.state_dict(), "bird_classifier.pth")
print("Modèle sauvegardé sous le nom 'bird_classifier.pth'")

# Charger le modèle sauvegardé (pour vérifier)
model.load_state_dict(torch.load("bird_classifier.pth"))
model.eval()  # Met le modèle en mode évaluation


Modèle sauvegardé sous le nom 'bird_classifier.pth'


C:\Users\paulj\AppData\Local\Temp\ipykernel_40712\3342411689.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("bird_classifier.pth"))


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=